# Process financial company data, from INPI

Resources:
* [documentation de la base SIRENE](https://www.sirene.fr/static-resources/htm/v_sommaire_311.htm#descvar)
* [comptes individuels vs consolidés](https://www.compta-online.com/comptes-individuels-comptes-sociaux-comptes-consolides-ao773)
* [documentation de l'API des comptes annuels, de l'INPI](https://www.inpi.fr/sites/default/files/documentation%20technique%20API_comptes_annuels%20v4_0.pdf)
    * quota journalier de 10000 requêtes, ou 10Go
* [source code processing this data](https://github.com/annuaire-entreprises-data-gouv-fr/search-infra/blob/main/workflows/data_pipelines/bilans_financiers/task_functions.py)

In [1]:
import pandas as pd
import hvplot.pandas
import numpy as np
from datetime import datetime

from src.data.make_dataset import DATA_PATH

pd.options.plotting.backend = "holoviews"

## load data

In [2]:
def get_fiscal_year(date):
    # Get the fiscal year based on the month of the date
    return date.year if date.month >= 7 else date.year - 1

current_fiscal_year = get_fiscal_year(datetime.now())

current_fiscal_year

2023

In [3]:
# CSV downloaded from
# https://www.data.gouv.fr/fr/datasets/r/9d213815-1649-4527-9eb4-427146ef2e5b

# another field (in the original INPI database, but not in this .csv) could
# be helpful : the duration of the "excercice comptable", since it is not always 1 year
fields = [
    "siren",
    "chiffre_d_affaires",
    "resultat_net",
    "date_cloture_exercice",
    "type_bilan",
]

df_bilan = pd.read_csv(
    DATA_PATH / 'raw/heavy/ratios_inpi_bce.csv',
    dtype=str,
    sep=";",
    usecols=fields,
)


In [4]:
df_bilan = df_bilan.rename(columns={"chiffre_d_affaires": "ca"})

# Convert columns to appropriate data types
df_bilan['date_cloture_exercice'] = pd.to_datetime(df_bilan['date_cloture_exercice'], format='%Y-%m-%d')
df_bilan["ca"] = df_bilan["ca"].astype(float)
df_bilan["resultat_net"] = df_bilan["resultat_net"].astype(float)

# Get the current fiscal year
df_bilan["annee_cloture_exercice"] = df_bilan["date_cloture_exercice"].apply(
    get_fiscal_year
)

In [5]:
len(df_bilan)

2714474

In [6]:
# Filter out rows with fiscal years greater than the current fiscal year
df_bilan = df_bilan[df_bilan["annee_cloture_exercice"] <= current_fiscal_year]

In [7]:
len(df_bilan)

2714473

In [8]:
# Drop duplicates based on siren, fiscal year, and type_bilan
df_bilan = df_bilan.drop_duplicates(
    subset=["siren", "annee_cloture_exercice", "type_bilan"], keep="last"
)

In [9]:
len(df_bilan)

2710711

In [10]:
df_bilan.head()

,siren,date_cloture_exercice,ca,resultat_net,type_bilan,annee_cloture_exercice
0,005620190,2022-08-31,7355288.0,104767.0,C,2022
1,005720164,2021-12-31,15837638.0,843095.0,C,2021
2,005750385,2019-12-31,1383353.0,57617.0,C,2019
3,005880737,2020-12-31,76690.0,42111.0,C,2020
4,006076434,2021-08-31,0.0,51935.0,C,2021


## look at the data

* 2.7M entries, including
    * 1.7M entries with date_cloture_exercice in december (the rest being mostly in march, june, or sept)
    * 980k entries with CA = 0.
    * 95k entries with resulta_net = 0.
    * 2M entries of type 'C', 630k entries of type 'S', and 13k entries of type 'K'
* 1M unique SIREN

In [7]:
df_bilan.columns

Index(['siren', 'date_cloture_exercice', 'ca', 'resultat_net', 'type_bilan',
       'annee_cloture_exercice'],
      dtype='object')

In [8]:
len(df_bilan)

2714474

In [9]:
df_bilan.siren.nunique()

1031868

In [10]:
df_bilan.ca.isna().sum()

np.int64(0)

In [11]:
df_bilan.ca.eq(0).sum()

np.int64(976932)

In [12]:
df_bilan.resultat_net.eq(0).sum()

np.int64(95838)

In [53]:
df_bilan['date_cloture_exercice'].dt.month.value_counts().plot(kind='bar')

:Bars   [date_cloture_exercice]   (count)

In [54]:
df_bilan.groupby('type_bilan')['siren'].count().plot(kind='bar')

:Bars   [type_bilan]   (siren)

In [66]:
np.log(df_bilan.ca.clip(lower=1.)).plot(kind='hist')

:Histogram   [ca]   (Count)

In [67]:
np.log(df_bilan.resultat_net.clip(lower=1.)).plot(kind='hist')

:Histogram   [resultat_net]   (Count)

## filter the data

In [11]:
df_bilan.groupby(["siren", "annee_cloture_exercice"])["type_bilan"].count().value_counts()

type_bilan
1    2684564
2      13060
3          9
Name: count, dtype: int64

In [12]:
df_bilan["type_bilan"].value_counts()

type_bilan
C    2063126
S     634135
K      13450
Name: count, dtype: int64

In [23]:
def keep_one_bilan(df_bilan, prio: dict):
    df_bilan["type_bilan_priority"] = df_bilan["type_bilan"].map(prio)
    df_bilan = df_bilan.sort_values(
        ["siren", "annee_cloture_exercice", "type_bilan_priority"], ascending=[True, False, True]
    )
    df_bilan = df_bilan.drop_duplicates(subset=["siren", "annee_cloture_exercice"], keep="first")
    df_bilan = df_bilan.drop(columns=["type_bilan_priority"])
    return df_bilan

In [24]:
df_bilan_consolidated_first = keep_one_bilan(df_bilan, {
    # Consolidated entries takes priority
    'K': '1-K',
    # Then complete entries
    'C': '2-C',
    # Then simplified entries
    'S': '3-S',
})

In [25]:
df_bilan_consolidated_first.columns

Index(['siren', 'date_cloture_exercice', 'ca', 'resultat_net', 'type_bilan',
       'annee_cloture_exercice'],
      dtype='object')

In [26]:
len(df_bilan_consolidated_first)

2697633

Good : we still have all 'K' entries

In [27]:
df_bilan_consolidated_first["type_bilan"].value_counts()

type_bilan
C    2051283
S     632900
K      13450
Name: count, dtype: int64

In [28]:
df_bilan_operational_first = keep_one_bilan(df_bilan, {
    # Complete entries take priority
    'C': '1-C',
    # Then simplified entries
    'S': '2-S',
    # Then consolidated entries
    'K': '3-K',
})

In [31]:
len(df_bilan_operational_first)

2697633

In [30]:
df_bilan_operational_first["type_bilan"].value_counts()

type_bilan
C    2063126
S     633170
K       1337
Name: count, dtype: int64

In [71]:
df = pd.merge(
    df_bilan_consolidated_first, 
    df_bilan_operational_first, 
    on=['siren', 'annee_cloture_exercice'], 
    suffixes=('_consolidated_first', '_operational_first')
)

In [84]:
x = df_bilan_consolidated_first.set_index(['siren', 'annee_cloture_exercice'])
y = df_bilan_operational_first.set_index(['siren', 'annee_cloture_exercice'])

In [85]:
is_diff = (x != y).agg(any, axis='columns')

In [86]:
x = x[is_diff]
y = y[is_diff]

In [87]:
len(x), len(y)

(12113, 12113)

All type bilans are differents : makes sense

In [88]:
(x != y).type_bilan.sum()

np.int64(12113)

Most resultat net / CA are differents : makes sense as well (the ones that are the same are probably zeros)

In [89]:
(x != y).resultat_net.sum()

np.int64(12082)

In [90]:
(x != y).ca.sum()

np.int64(11930)

A couple have a different date_cloture_exercice, this is annoying

In [91]:
(x != y).date_cloture_exercice.sum()

np.int64(57)

In [92]:
z = df.set_index(['siren', 'annee_cloture_exercice'])[is_diff]

In [93]:
z[(x != y).date_cloture_exercice].head()

date_cloture_exercice_consolidated_first  \
siren     annee_cloture_exercice                                            
055812440 2021                                                 2021-08-31   
          2020                                                 2020-08-31   
          2019                                                 2019-08-31   
305523664 2020                                                 2020-12-31   
326300969 2019                                                 2020-03-19   

                                  ca_consolidated_first  \
siren     annee_cloture_exercice                          
055812440 2021                             1.742800e+10   
          2020                             1.932200e+10   
          2019                             2.195400e+10   
305523664 2020                             0.000000e+00   
326300969 2019                             2.553600e+07   

                                  resultat_net_consolidated_first  \
siren     annee_cloture_exercice                                    
055812440 2021                                        114000000.0   
          2020                                       -336000000.0   
          2019                                       -673000000.0   
305523664 2020                                                0.0   
326300969 2019                                         -6894000.0   

                                 type_bilan_consolidated_first  \
siren     annee_cloture_exercice                                 
055812440 2021                                               K   
          2020                                               K   
          2019                                               K   
305523664 2020                                               K   
326300969 2019                                               K   

                                 date_cloture_exercice_operational_first  \
siren     annee_cloture_exercice                                           
055812440 2021                                                2021-12-31   
          2020                                                2020-12-31   
          2019                                                2019-12-31   
305523664 2020                                                2021-06-30   
326300969 2019                                                2020-03-31   

                                  ca_operational_first  \
siren     annee_cloture_exercice                         
055812440 2021                               3170000.0   
          2020                               1984000.0   
          2019                               3052000.0   
305523664 2020                                102851.0   
326300969 2019                               1984000.0   

                                  resultat_net_operational_first  \
siren     annee_cloture_exercice                                   
055812440 2021                                       110527000.0   
          2020                                       160734000.0   
          2019                                       153861000.0   
305523664 2020                                        72224620.0   
326300969 2019                                        -5162000.0   

                                 type_bilan_operational_first  
siren     annee_cloture_exercice                               
055812440 2021                                              C  
          2020                                              S  
          2019                                              C  
305523664 2020                                              C  
326300969 2019                                              C

In [70]:
df_bilan_consolidated_first.to_csv(DATA_PATH / 'processed/synthese_bilans_financiers.csv')